In [1]:
import os
import  keras
import  numpy  as  np
import  tensorflow  as  tf
import  matplotlib.pyplot  as  plt
from  keras  import  backend  as  k
from tensorflow.keras.preprocessing import image
from  keras.applications.inception_v3  import  InceptionV3, decode_predictions

tf.compat.v1.disable_eager_execution ()

ModuleNotFoundError: No module named 'keras'

In [ ]:
data_dir = os.path.dirname(__file__) + "/"
iv3 = InceptionV3()

In [ ]:
# iv3.summary()

In [ ]:
beer = 'beer.jpg'
stop = 'Stop.png'
hackImage = 'hackImage.png'
img = image.img_to_array(image.load_img(data_dir + hackImage, target_size=(299, 299)))
print(img)

Si se imprime img los resultados van a ser de 0 a 255 los colores pero incepcion funciona de -1 a 1 para lo cual se va a reescalar la imagen

In [ ]:
# Conversión de 0-255 a 0-1
img /= 255
img -= 0.5
img *= 2

In [ ]:
# El modelo pide que entre un tensor con una dimension mas esa dimension extra es el tamaño del batch (+1 imágen)
img = img.reshape([1, img.shape[0], img.shape[1], img.shape[2]])
print (img.shape)

In [ ]:
# Predicción
y=iv3.predict(img)
decode_predictions(y)


## Ataque adversario

In [ ]:
# Vamos a construir un nuevo grapho que permita clasificar las imágenes.
inp_layer = iv3.layers[0].input
out_layer = iv3.layers[-1].output
target_class = 951 # Limón
loss = out_layer[0, target_class]
grad = k.gradients(loss, inp_layer)[0]

# Que calcule el grafo de gradientes entre el tensor y la funcion de coste #tensor que dice como variar los pixeles
optimize_gradient = k.function([inp_layer, k.learning_phase()], [grad, loss])


In [ ]:

pert=0.009
max_pert=img+pert
min_pert=img-pert
adv = np.copy(img)
cost = 0.0
while cost<0.95:
    gr, cost = optimize_gradient([adv,0]) #0 es que estamos en modo testing
    adv += gr #sumarle los pixeles para que se vea manipulado
    adv=np.clip(adv,min_pert,max_pert)#hace que la imagen se encuentre en el rango de perturbacion
    print("target cost: ", cost) #que imprima el coste